In [1]:
import tweepy
import json
import pandas as pd
import random
from dotenv import load_dotenv
import os
import time
from datetime import datetime
load_dotenv()

True

# Sentiment analysis de tweets respecto al plebiscito en Septiembre 2022 Chile

Este script busca en tweeter, con la los privilegios de la cuenta develop gratuita, los tweets según las palabras clave, la localización y las fechas determinadas.

La información mas importante de cada tweet es guardado en un dataframe.

La API libre de twitter tiene varias limitaciones, entre ella la fecha:

    Keep in mind that the search index has a 7-day limit. In other words, no tweets will be found for a date older than one week.
    
No se puede traer tweets mas antiguos que una semana. Ademas del peso sobre la cantidad de tweets, que cuelga la API por 15 minutos

##### Definición de palabras clave (_keywords_) a buscar dentro del texto de los tweets:

In [ ]:
keywords_list = ['Constituyente',
'Convención Constituyente',
'Convención Constitucional',
'Asamblea Constituyente',
'plebiscito',
'referéndum',
'constitución chilena',
'Acuerdo por la paz social y la Nueva Constitución',
'Vamos por Chile',
'Unión Demócrata Independiente',
'Constituyentes Apartidistas',
'Independientes No Neutrales',
'Lista del Pueblo',
'Apruebo Dignidad',
'Lista del Apruebo',
'Pueblo Constituyente']

keywords_list_II = ['Chile Digno',
'Chile Unido',
'Unidos por Chile'
'Chile Libre',
'Coordinadora Constituyente Plurinacional y Popular',
'Somos región',
'Colectivo Socialista',
'Independientes por una Nueva Constitución',
'Movimientos sociales constituyentes',
'Colectivo del Apruebo',
'Wallmapur',
'Elisa Loncón',
'Campaña de desprestigio',
'rechazo del prebiscito de 2020',
'Carmen Gloria Valladares Moyano',
'Elisa Loncon',
'Destitucion de Elisa Loncon']

In [ ]:
keywords_list = ["(" + word + ")" for word in keywords_list]

'#DestitucionDeElisaLoncon',
'juicio político',
'Jaime Bassa',
'Marcela Cubillos',
'Jaime Mañalich',
'Autodesprestigio',
'María Elisa Quinteros Cáceres',
'Gaspar Dominguez Donoso']

In [ ]:
keywords_list  = ' OR '.join(keywords_list)

In [ ]:
keywords_list

#### Funciones
El script tiene dos funciones principales:
- Activación de la API con las credenciales
- Búsqueda de los tweets con lo parámetros definidos

##### Función Activación

In [3]:
# Función de generación de la API
def oauth():
    #Pass in our twitter API authentication key
    consumer_key = os.getenv('CONSUMER_KEY')
    consumer_secret = os.getenv('CONSUMER_SECRET')
    access_token = os.getenv('ACCESS_TOKEN')
    access_token_secret = os.getenv('ACCESS_TOKEN_SECRET')
    
    auth = tweepy.OAuth1UserHandler(
        consumer_key, consumer_secret,
        access_token, access_token_secret
    )
    #Instantiate the tweepy API
    api = tweepy.API(auth, wait_on_rate_limit=True)
    return api

##### Función Búsqueda

In [ ]:
# Función de búsqueda de tweets según los keywords, cantidad y localización
def search(geo, serch_query, no_of_tweets):
    try:
        #The number of tweets we want to retrieved from the search
        tweets = api.search_tweets(
                                   q=serch_query, 
                                   geocode = geo, 
                                   count=no_of_tweets
                                  )
        #Pulling Some attributes from the tweet
        attributes_container = [[tweet.user.name, 
                                 tweet.created_at, 
                                 tweet.favorite_count, 
                                 tweet.source,  
                                 tweet.text 
                                ] for tweet in tweets]
        # Creamos las keys del dict para guardar la info de la lista attributes_containter
        keys = ["User", 
                "Date Created", 
                "Number of Likes", 
                "Source of Tweet", 
                "Tweet"]
        # Generamos el dict vacío
        dict = {}
        # Variable que se incrementa cada vez que analizamos un twit nuevo, y genera un indice en el dict
        # para diferenciarlo del anterior
        a=1
        
        # Pasamos el resultado de la búsqueda a un dict, que es lo que devuelve la función
        # para que en el código debajo de la llamada lo convierta en DF
        for value in tweets:
            dict[a] = {}
            dict[a]['User'] = value.user.name
            dict[a]['Date Created'] = value.created_at
            dict[a]['Number of likes'] = value.favorite_count
            dict[a]['Source of Tweet'] = value.source
            dict[a]['Tweet'] = value.text
            a = a+1
        df = pd.DataFrame()
        df = pd.DataFrame.from_dict(dict,orient='index')
        return df
    except BaseException as e:
        print('Status Failed On,',str(e))
        return

#### Ejecución 

##### Instanciamos la API

api = oauth()

#####  Cargamos los parámetros de búsqueda


- `sgo`: latitud y longitud de Santiago de Chile
- `radio`: radio en km alrededor del punto
- `no_of_tweets`: Cantidad de tweets para cada keyword
- `geo`: arma el feature según lo cargado anteriormente, con la ciudad y el radio.
- `keywords_list`: Se incluye en esta sección sólo para testear. Se limita a dos palabras. Si se comenta esa línea, usa toda la lista presentada arriba)

In [ ]:
sgo = '-33.4489,-70.6693'
radio = '100km'
no_of_tweets =100000
geo = str(sgo) + ',' + str(radio) 
# keywords_list = ['Chile'] # Comentar esta linea para usar toda la keywords_list
df = pd.DataFrame()

#####  Lanzamos la búsqueda

In [ ]:
# df = pd.DataFrame()
df = search(geo, 
              "Chile", 
              no_of_tweets)


In [ ]:
df

In [ ]:
# pd.options.display.max_colwidth = 240
df

##### Iteración por lista de keywords, no recomendada

In [ ]:
for search_query in keywords_list:
    dict = search(geo, keywords_list, no_of_tweets)
    df1 =pd.DataFrame.from_dict(dict,orient='index')
    frames = [df, df1]
    df = pd.concat(frames)
#     time.sleep(910) # Esperamos 15 minutos + 10 segundo para no reventar la API

#####  Mostramos el DF 

In [ ]:
df

#####  Guardamos el dF

In [ ]:
filename = 'plebiscito_' + datetime.now().strftime('%m_%d_%Y_%H:%M:%S') + '.csv'
df.to_csv(filename, index = False)

# Pagination con reasearch tokens

In [4]:
sgo = '-33.4489,-70.6693'
radio = '100km'
no_of_tweets =100000
geo = str(sgo) + ',' + str(radio)

# api = oauth()

In [6]:
# Set the parameters for the search
search = '(Convención Constituyente) OR (referéndum)'
# Set the number of pages
limit=2
# Counter for printing the page number
page_no = 1

# Create columns for the DataFrame
data = []
columns = ['Time','User', 'Tweet']
# Search using pagination
pages = tweepy.Cursor(api.search_tweets,q=search, tweet_mode='extended', geocode = geo).pages(limit)

# Iterate through the pages and print the text of the tweets
for page in pages:    
    for i in range(len(page)):
        data.append([page[i].created_at, page[i].user.screen_name, page[i].full_text])
    page_no += 1

df = pd.DataFrame(data, columns=columns)


In [7]:
df

,Time,User,Tweet
0,2022-08-24 16:37:44+00:00,MarioElias77,@gonzalo_sanchez @thecliniccl @bnavarroz El re...
1,2022-08-24 16:23:29+00:00,ReviewerRandom,@Michellopezy @LuisBahamondes Pues adivina: la...
2,2022-08-24 15:53:40+00:00,MarcoMira20,@Gusano_Rastrero A mí me parece bien el referé...
3,2022-08-24 14:57:55+00:00,DaphneLack,No se requiere convocar a una nueva CC. Solo c...
4,2022-08-24 14:20:21+00:00,aRRRaya,La minuta amarillista e hipócrita política de ...
5,2022-08-24 05:42:39+00:00,Otro_criticon,@SebastianRochaZ Lo que no cuenta este persona...
6,2022-08-24 03:57:45+00:00,recampchile,@Rodrigo16427424 @FrutillaLibert2 @gabrielbori...
7,2022-08-24 03:15:35+00:00,Vlathymir,@HernanLarrain Sea preciso con la información....
8,2022-08-24 03:14:04+00:00,_renzuc,Se acuerdan de la convención constituyente? Pr...
9,2022-08-24 03:07:23+00:00,wchgrafica,"Legítimo: Acuerdo, elección de convencionales,..."
